In [1]:
import sys, os


import logging
import pandas as pd
import numpy as np
from pathlib import Path
# sys.path.insert(0,'..')
project_dir = str(Path(os.getcwd()).parent)
if not project_dir in sys.path:
    sys.path.insert(0,project_dir)
sys.path
from BankClassify import BankClassify, assign_target_account, assign_label
from load_data import read_dkb_csv, read_paypal_csv

In [2]:
%load_ext autoreload

%autoreload 2

In [3]:
logging.basicConfig(level=logging.DEBUG)

In [4]:
input_data_path =  Path('/Users/jangie/Documents/Finanzen/Detailed_Data')

In [5]:
# bc.data['dkb_3109'].index = len(bc.data['dkb_3109'])

In [12]:
bc = BankClassify(Path('../data'))

DEBUG:root:FFFFF /Users/jangie/Projects/DataScience/BankClassify/BankClassify.py
INFO:root:loaded previous dkb_3109 data 541 entries
INFO:root:loaded previous dkb_4217 data 1212 entries
INFO:root:train dataset size 1324


In [13]:
bc.data_all

datetime   amount  \
dkb_3109 0    2021-12-29   -39.99   
         1    2021-12-27   -57.00   
         2    2021-12-27   -16.79   
         3    2021-12-23  3845.20   
         4    2021-12-22   500.00   
...                  ...      ...   
dkb_4217 1203 2022-01-21   -17.25   
         1204 2022-01-21   -21.95   
         1205 2022-01-20   -40.96   
         1206 2022-01-20    -8.22   
         1211 2022-01-07   -19.50   

                                                            desc  account_nr  \
dkb_3109 0     Lastschrift Vodafone Deutschland GmbH 12/2021 ...  1002423109   
         1     Überweisung EDITH ESPINOZA DE THIEKE Rechn.Nr....  1002423109   
         2     Lastschrift Vodafone GmbH 0000114111198 001508...  1002423109   
         3     Lohn, Gehalt, Rente IAV GmbH Ingenieurgesellsc...  1002423109   
         4     Gutschrift DR.ST. U. C. GIESELER Feliz natal p...  1002423109   
...                                                          ...         ...   
dkb_4217 1203  Kartenzahlung/-abrechnung DM FIL.2814 H:65273/...  1071944217   
         1204  Kartenzahlung/-abrechnung HUGENDUBEL 0665 FIL....  1071944217   
         1205  Kartenzahlung/-abrechnung EDEKA ZAHL, BERLIN//...  1071944217   
         1206  Kartenzahlung/-abrechnung HANSEN OBST GBR GIR ...  1071944217   
         1211  Lastschrift SWAPFIETS BY BUCKAROO Swapfiets: D...  1071944217   

              target account                               class  
dkb_3109 0               NaN               Bill - Communications  
         1               NaN       Bill - Utilities, Rent & Fees  
         2               NaN               Bill - Communications  
         3               NaN          Income - Salary & Benefits  
         4               NaN                   Income - Presents  
...                      ...                                 ...  
dkb_4217 1203            NaN  Supermarket & Everyday commodities  
         1204            NaN                               Other  
         1205            NaN  Supermarket & Everyday commodities  
         1206            NaN  Supermarket & Everyday commodities  
         1211            NaN                    Public Transport  

[1324 rows x 6 columns]

In [ ]:
for f in Path('../data').glob('*.csv'):
    print(f.with_suffix('').name)

In [ ]:
# bc.data

In [ ]:
sorted([f for f in (input_data_path/'dkb').glob('*.csv')])

### load new raw data and add to existing dataset

#### paypal data

In [7]:
filename = Path('/Users/jangie/Documents/Finanzen/Detailed_Data/Paypal/Paypal_01 Jan 2019 - 29 Dec 2022.CSV')

In [8]:
df_pp = read_paypal_csv(filename)
len(df_pp)

DEBUG:root:loaded 1382 paypal records
DEBUG:root:1084 paypal records after dropping `Balance Impact` = `Memo` entries


1084

In [9]:
df_pp['Type'].value_counts()

Bank Deposit to PP Account                426
Express Checkout Payment                  252
PreApproved Payment Bill User Payment     207
General Card Deposit                       59
General Payment                            39
Mobile Payment                             22
General Withdrawal                         21
Payment Refund                             21
General Currency Conversion                14
Account Hold for Open Authorization         5
Reversal of General Account Hold            5
Donation Payment                            4
Reversal of ACH Deposit                     3
Website Payment                             2
Reversal of ACH Withdrawal Transaction      2
Payment Reversal                            1
General Card Withdrawal                     1
Name: Type, dtype: Int64

In [41]:
df_pp

,datetime,desc,Date,Time,TimeZone,Name,Type,Status,Currency,amount,...,Transaction Event Code,Payment Tracking ID,Bank Reference ID,Country Code,Balance Impact,Invoice Number.1,PO Number,Customer Reference Number,Payflow Transaction ID (PNREF),Original Invoice ID
Transaction ID,,,,,,,,,,,,,,,,,,,,,
4WY592183V7454844-27052019 07:02:28+0000,2019-05-27 07:02:28+00:00,Alexander Kihm Alles Liebe und viel Spass!! ...,27/05/2019,09:02:28,+02:00,Alexander Kihm,General Payment,Completed,EUR,-100.00,...,T0000,<NA>,<NA>,<NA>,Debit,<NA>,<NA>,<NA>,<NA>,<NA>
1CC51055UW323635G-03092020 08:31:34+0000,2020-09-03 08:31:34+00:00,Konrad Exner �,03/09/2020,10:31:34,+02:00,Konrad Exner,Mobile Payment,Completed,EUR,6.00,...,T0011,<NA>,<NA>,<NA>,Credit,<NA>,<NA>,<NA>,<NA>,<NA>
36F184282C430600K-26092020 06:53:18+0000,2020-09-26 06:53:18+00:00,DURSTEXPRESS GmbH Germany Bestellung 10585-247...,26/09/2020,08:53:18,+02:00,DURSTEXPRESS GmbH,Express Checkout Payment,Completed,EUR,-98.22,...,T0006,<NA>,<NA>,DE,Debit,<NA>,<NA>,<NA>,<NA>,<NA>
22149102YJ0036701-26092020 06:53:18+0000,2020-09-26 06:53:18+00:00,Bestellung 10585-24705-37C7E9,26/09/2020,08:53:18,+02:00,<NA>,Bank Deposit to PP Account,Pending,EUR,89.09,...,T0300,<NA>,5CX23UU8FFZ7A,<NA>,Credit,<NA>,<NA>,<NA>,<NA>,<NA>
9UY72313YA366071V-26092020 10:40:59+0000,2020-09-26 10:40:59+00:00,DURSTEXPRESS GmbH Bestellung 10585-24705-37C7E9,26/09/2020,12:40:59,+02:00,DURSTEXPRESS GmbH,Payment Refund,Completed,EUR,9.70,...,T1107,<NA>,<NA>,<NA>,Credit,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8N691565N7322580S-10122022 12:16:36+0000,2022-12-10 12:16:36+00:00,Nike Order,10/12/2022,13:16:36,+01:00,<NA>,General Withdrawal,Completed,EUR,-33.97,...,T0400,<NA>,YYW1024024816919,<NA>,Debit,<NA>,<NA>,<NA>,<NA>,C01133505577
4TG0164811967221U-10122022 18:24:47+0000,2022-12-10 18:24:47+00:00,app smart GmbH / order smart Germany Salmon Lo...,10/12/2022,19:24:47,+01:00,app smart GmbH / order smart,Express Checkout Payment,Completed,EUR,-34.15,...,T0006,<NA>,<NA>,DE,Debit,<NA>,<NA>,<NA>,<NA>,6394ceb8a48fd
82B52026993747919-10122022 18:24:47+0000,2022-12-10 18:24:47+00:00,Salmon Lover Menü - normal,10/12/2022,19:24:47,+01:00,<NA>,Bank Deposit to PP Account,Pending,EUR,34.15,...,T0300,<NA>,1024030996502,<NA>,Credit,<NA>,<NA>,<NA>,<NA>,6394ceb8a48fd


In [25]:
data = pd.read_csv(filename, decimal=",", dtype="string")

In [27]:
# data

#### dkb data

In [43]:
filename = Path('/Users/jangie/Documents/Finanzen/Detailed_Data/dkb/1002423109.csv')

In [44]:
df_new = read_dkb_csv(filename)

INFO:root:number of data loaded: 70
DEBUG:root:new data from /Users/jangie/Documents/Finanzen/Detailed_Data/dkb/1002423109.csv with 70 entries, years [2022]


In [46]:
df_new['date']

0    2022-09-30
1    2022-09-30
2    2022-09-29
3    2022-09-28
4    2022-09-27
        ...    
65   2022-08-04
66   2022-08-03
67   2022-08-03
68   2022-08-02
69   2022-08-01
Name: date, Length: 70, dtype: datetime64[ns]

In [11]:
len(bc.data['dkb_3109'][bc.data['dkb_3109']['class'].isna()])

123

In [12]:
for k, v in bc.data.items():
    print(f"{k}: {len(v)} entires")

dkb_3109: 541 entires
dkb_4217: 1212 entires


In [13]:
df_new

,date,amount,desc,account_nr
0,2022-09-30,-315.64,Kartenzahlung/-abrechnung VIVA RESORT PROT.PRI...,1002423109
1,2022-09-30,-4.00,Kartenzahlung/-abrechnung H.RESORT SUPERMERCAD...,1002423109
2,2022-09-29,-39.99,Lastschrift Vodafone Deutschland GmbH 09/2022 ...,1002423109
3,2022-09-28,4157.32,"Lohn, Gehalt, Rente IAV GmbH Ingenieurgesellsc...",1002423109
4,2022-09-27,-69.00,Überweisung GISELA SILVA Rechnungs-Nr.: 142/20...,1002423109
...,...,...,...,...
65,2022-08-04,-40.00,Kartenzahlung/-abrechnung RESTAURANT SONNE GIR...,1002423109
66,2022-08-03,-100.00,Lastschrift European Bank for Financial Servic...,1002423109
67,2022-08-03,-23.00,Lastschrift GIRASOLES - SONNENBLUMEN Beitrag N...,1002423109
68,2022-08-02,-7.20,Kartenzahlung/-abrechnung Lagard re Travel Ret...,1002423109


In [14]:
bc.add_data(df_new, f"dkb_{filename.with_suffix('').name[-4:]}")

DEBUG:root:adding dkb_3109 data!
DEBUG:root:previous dataset 541 entries
DEBUG:root:new dataset 70 entries with 0 new ones


In [18]:
bc.data_unlabeled

date    amount  \
dkb_3109 477 2022-09-30   -315.64   
         478 2022-09-30     -4.00   
         479 2022-09-29    -39.99   
         480 2022-09-28   4157.32   
         481 2022-09-27    -69.00   
         483 2022-09-26    -29.84   
         484 2022-09-26    -19.99   
         485 2022-09-22    -48.00   
         486 2022-09-21  -1000.00   
         487 2022-09-21  -1000.00   
         488 2022-09-20    -90.00   
         489 2022-09-19    -48.00   
         490 2022-09-19    -42.80   
         491 2022-09-19    -18.59   
         492 2022-09-19    -11.94   
         493 2022-09-19     -5.00   
         495 2022-09-12    -34.00   
         496 2022-09-12    -26.00   
         497 2022-09-09    -48.00   
         498 2022-09-09     -5.69   
         499 2022-09-08     -3.95   
         500 2022-09-08     -1.44   
         501 2022-09-07    -65.95   
         502 2022-09-06   -100.00   
         503 2022-09-06    -61.00   
         504 2022-09-06    -22.59   
         505 2022-09-06    -12.60   
         506 2022-09-06     -4.90   
         507 2022-09-05   -100.00   
         508 2022-09-05    -23.00   
         509 2022-09-01  -2450.50   
         510 2022-09-01    -48.00   
         511 2022-09-01    -48.00   
         512 2022-08-31 -20000.00   
         513 2022-08-30 -20000.00   
         514 2022-08-30    -39.00   
         515 2022-08-30     -9.30   
         516 2022-08-30   3983.32   
         517 2022-08-29    -39.99   
         518 2022-08-29     -7.00   
         519 2022-08-26 -25000.00   
         520 2022-08-25    -19.99   
         522 2022-08-24    -38.00   
         523 2022-08-24     -3.80   
         525 2022-08-22  -1000.00   
         526 2022-08-22  -1000.00   
         527 2022-08-22    -90.00   
         528 2022-08-22    -56.58   
         529 2022-08-22    -48.00   
         531 2022-08-19    150.00   
         532 2022-08-18     -2.44   
         533 2022-08-16    -17.80   
         534 2022-08-15    -55.08   
         535 2022-08-15    -48.00   
         536 2022-08-15    -29.05   
         539 2022-08-08   -116.01   
         540 2022-08-08    -61.00   

                                                           desc  account_nr  \
dkb_3109 477  Kartenzahlung/-abrechnung VIVA RESORT PROT.PRI...  1002423109   
         478  Kartenzahlung/-abrechnung H.RESORT SUPERMERCAD...  1002423109   
         479  Lastschrift Vodafone Deutschland GmbH 09/2022 ...  1002423109   
         480  Lohn, Gehalt, Rente IAV GmbH Ingenieurgesellsc...  1002423109   
         481  Überweisung GISELA SILVA Rechnungs-Nr.: 142/20...  1002423109   
         483  Kartenzahlung/-abrechnung EDEKA GRACKA, BERLIN...  1002423109   
         484  Lastschrift Vodafone GmbH 0000114111198 001835...  1002423109   
         485  Überweisung GISELA SILVA Rechnungs-Nr.: 138/20...  1002423109   
         486  Lastschrift Sauren SICAV SPARPLAN ISIN LU01155...  1002423109   
         487  Lastschrift Sauren SICAV SPARPLAN ISIN LU17170...  1002423109   
         488  Dauerauftrag GIRASOLES-FOERDERVEREIN E. V.  I....  1002423109   
         489  Überweisung GISELA SILVA Rechnungs-Nr.: 133/20...  1002423109   
         490  Kartenzahlung/-abrechnung ALDI SAGT DANKE 28 0...  1002423109   
         491  Kartenzahlung/-abrechnung ASIL GMBH//BERLIN/DE...  1002423109   
         492  Kartenzahlung/-abrechnung Atlantis-Apotheke//B...  1002423109   
         493  Kartenzahlung/-abrechnung BONG APOTHEKE//BERLI...  1002423109   
         495  Kartenzahlung/-abrechnung VINOTRIA GMBH CO KG/...  1002423109   
         496  Kartenzahlung/-abrechnung BEZIRKSAMT CHARLOTTE...  1002423109   
         497  Überweisung GISELA SILVA Rechnungs-Nr.: 129/20...  1002423109   
         498  Kartenzahlung/-abrechnung Burger King 15800//N...  1002423109   
         499  Kartenzahlung/-abrechnung Der Beck GmbH Filial...  1002423109   
         500  Kartenzahlung/-abrechnung MUELLER SAGT DANKE//...  1002423109   
         501  Kartenzahlung/-abrechnung DECATHLON Deuts

### assign target account

In [16]:
r'PAYPAL (EUROPE) S.A.R.L .*'

'PAYPAL (EUROPE) S.A.R.L .*'

In [17]:
df = bc.data_unlabeled

In [18]:
# df[df['desc'].str.match(r'^Gutschrift J.*und C.*')]

In [19]:
# bc.data_unlabeled[bc.data_unlabeled['amount']> 2000]

In [20]:
df_update = assign_target_account(bc.data_unlabeled)
df_update

DEBUG:root:assigning 5 entries to dkb_4217


>>>>PAYPAL (EUROPE) S.A.R.L .*
>>>>^Überweisung HERR .*
>>>>Umbuchung KREDITKARTEN GELDANLAGE
>>>>^Gutschrift J.*und C.*


date   amount  \
dkb_3109 477 2022-09-30  -315.64   
         478 2022-09-30    -4.00   
         479 2022-09-29   -39.99   
         480 2022-09-28  4157.32   
         481 2022-09-27   -69.00   
...                 ...      ...   
         534 2022-08-15   -55.08   
         535 2022-08-15   -48.00   
         536 2022-08-15   -29.05   
         539 2022-08-08  -116.01   
         540 2022-08-08   -61.00   

                                                           desc  account_nr  \
dkb_3109 477  Kartenzahlung/-abrechnung VIVA RESORT PROT.PRI...  1002423109   
         478  Kartenzahlung/-abrechnung H.RESORT SUPERMERCAD...  1002423109   
         479  Lastschrift Vodafone Deutschland GmbH 09/2022 ...  1002423109   
         480  Lohn, Gehalt, Rente IAV GmbH Ingenieurgesellsc...  1002423109   
         481  Überweisung GISELA SILVA Rechnungs-Nr.: 142/20...  1002423109   
...                                                         ...         ...   
         534  Lastschrift Rundfunk ARD, ZDF, DRadio Rundfunk...  1002423109   
         535  Überweisung GISELA SILVA Rechnungs-Nr.: 100/20...  1002423109   
         536  Kartenzahlung/-abrechnung ROSSMANN VIELEN DANK...  1002423109   
         539  Kartenzahlung/-abrechnung AMBROSINO SUPERMARKT...  1002423109   
         540  Lastschrift VATTENFALL EUROPE SALES S/83663436...  1002423109   

             target account class  
dkb_3109 477            NaN   NaN  
         478            NaN   NaN  
         479            NaN   NaN  
         480            NaN   NaN  
         481            NaN   NaN  
...                     ...   ...  
         534            NaN   NaN  
         535            NaN   NaN  
         536            NaN   NaN  
         539            NaN   NaN  
         540            NaN   NaN  

[62 rows x 6 columns]

In [21]:
df_update[~df_update['target account'].isna()]

date   amount  \
dkb_3109 482 2022-09-27   5000.0   
         494 2022-09-19   3000.0   
         521 2022-08-25  20000.0   
         524 2022-08-23  20000.0   
         530 2022-08-22  20000.0   

                                                           desc  account_nr  \
dkb_3109 482  Gutschrift Jan Gieseler und Camila Lobo Costa ...  1002423109   
         494  Gutschrift Jan Gieseler und Camila Lobo Costa ...  1002423109   
         521  Gutschrift Jan Gieseler und Camila Lobo Costa ...  1002423109   
         524  Gutschrift Jan Gieseler und Camila Lobo Costa ...  1002423109   
         530  Gutschrift Jan Gieseler und Camila Lobo Costa ...  1002423109   

             target account class  
dkb_3109 482       dkb_4217   NaN  
         494       dkb_4217   NaN  
         521       dkb_4217   NaN  
         524       dkb_4217   NaN  
         530       dkb_4217   NaN

In [22]:
# df_update.loc['dkb_3109']

In [23]:
bc.data.keys()

dict_keys(['dkb_3109', 'dkb_4217'])

In [24]:
bc.update_data(df_update)

DEBUG:root:updating dataset_name: dkb_3109


ffff Index(['date', 'amount', 'desc', 'account_nr', 'target account', 'class'], dtype='object')


In [29]:
len(bc.data_unlabeled)

57

In [30]:
bc.save_data()

### label that data with regex

In [19]:
df_update = assign_label(bc.data_unlabeled)
print(len(df_update))
df_update

DEBUG:root:labeling 2 entries as 'Personal Care & Sport'
DEBUG:root:labeling 2 entries as 'Education'
DEBUG:root:labeling 1 entries as 'Supermarket & Everyday commodities'
DEBUG:root:labeling 1 entries as 'Restaurants & Bars'
DEBUG:root:labeling 1 entries as 'Supermarket & Everyday commodities'
DEBUG:root:labeling 1 entries as 'Supermarket & Everyday commodities'
DEBUG:root:labeling 1 entries as 'Supermarket & Everyday commodities'
DEBUG:root:labeling 1 entries as 'Bill - Utilities, Rent & Fees'
DEBUG:root:labeling 5 entries as 'Investments'
DEBUG:root:labeling 4 entries as 'Investments'
DEBUG:root:labeling 2 entries as 'Bill - Utilities, Rent & Fees'
DEBUG:root:labeling 4 entries as 'Bill - Communications'
DEBUG:root:labeling 2 entries as 'Income - Salary & Benefits'
DEBUG:root:labeling 8 entries as 'Education'


57


date    amount  \
dkb_3109 477 2022-09-30   -315.64   
         478 2022-09-30     -4.00   
         479 2022-09-29    -39.99   
         480 2022-09-28   4157.32   
         481 2022-09-27    -69.00   
         483 2022-09-26    -29.84   
         484 2022-09-26    -19.99   
         485 2022-09-22    -48.00   
         486 2022-09-21  -1000.00   
         487 2022-09-21  -1000.00   
         488 2022-09-20    -90.00   
         489 2022-09-19    -48.00   
         490 2022-09-19    -42.80   
         491 2022-09-19    -18.59   
         492 2022-09-19    -11.94   
         493 2022-09-19     -5.00   
         495 2022-09-12    -34.00   
         496 2022-09-12    -26.00   
         497 2022-09-09    -48.00   
         498 2022-09-09     -5.69   
         499 2022-09-08     -3.95   
         500 2022-09-08     -1.44   
         501 2022-09-07    -65.95   
         502 2022-09-06   -100.00   
         503 2022-09-06    -61.00   
         504 2022-09-06    -22.59   
         505 2022-09-06    -12.60   
         506 2022-09-06     -4.90   
         507 2022-09-05   -100.00   
         508 2022-09-05    -23.00   
         509 2022-09-01  -2450.50   
         510 2022-09-01    -48.00   
         511 2022-09-01    -48.00   
         512 2022-08-31 -20000.00   
         513 2022-08-30 -20000.00   
         514 2022-08-30    -39.00   
         515 2022-08-30     -9.30   
         516 2022-08-30   3983.32   
         517 2022-08-29    -39.99   
         518 2022-08-29     -7.00   
         519 2022-08-26 -25000.00   
         520 2022-08-25    -19.99   
         522 2022-08-24    -38.00   
         523 2022-08-24     -3.80   
         525 2022-08-22  -1000.00   
         526 2022-08-22  -1000.00   
         527 2022-08-22    -90.00   
         528 2022-08-22    -56.58   
         529 2022-08-22    -48.00   
         531 2022-08-19    150.00   
         532 2022-08-18     -2.44   
         533 2022-08-16    -17.80   
         534 2022-08-15    -55.08   
         535 2022-08-15    -48.00   
         536 2022-08-15    -29.05   
         539 2022-08-08   -116.01   
         540 2022-08-08    -61.00   

                                                           desc  account_nr  \
dkb_3109 477  Kartenzahlung/-abrechnung VIVA RESORT PROT.PRI...  1002423109   
         478  Kartenzahlung/-abrechnung H.RESORT SUPERMERCAD...  1002423109   
         479  Lastschrift Vodafone Deutschland GmbH 09/2022 ...  1002423109   
         480  Lohn, Gehalt, Rente IAV GmbH Ingenieurgesellsc...  1002423109   
         481  Überweisung GISELA SILVA Rechnungs-Nr.: 142/20...  1002423109   
         483  Kartenzahlung/-abrechnung EDEKA GRACKA, BERLIN...  1002423109   
         484  Lastschrift Vodafone GmbH 0000114111198 001835...  1002423109   
         485  Überweisung GISELA SILVA Rechnungs-Nr.: 138/20...  1002423109   
         486  Lastschrift Sauren SICAV SPARPLAN ISIN LU01155...  1002423109   
         487  Lastschrift Sauren SICAV SPARPLAN ISIN LU17170...  1002423109   
         488  Dauerauftrag GIRASOLES-FOERDERVEREIN E. V.  I....  1002423109   
         489  Überweisung GISELA SILVA Rechnungs-Nr.: 133/20...  1002423109   
         490  Kartenzahlung/-abrechnung ALDI SAGT DANKE 28 0...  1002423109   
         491  Kartenzahlung/-abrechnung ASIL GMBH//BERLIN/DE...  1002423109   
         492  Kartenzahlung/-abrechnung Atlantis-Apotheke//B...  1002423109   
         493  Kartenzahlung/-abrechnung BONG APOTHEKE//BERLI...  1002423109   
         495  Kartenzahlung/-abrechnung VINOTRIA GMBH CO KG/...  1002423109   
         496  Kartenzahlung/-abrechnung BEZIRKSAMT CHARLOTTE...  1002423109   
         497  Überweisung GISELA SILVA Rechnungs-Nr.: 129/20...  1002423109   
         498  Kartenzahlung/-abrechnung Burger King 15800//N...  1002423109   
         499  Kartenzahlung/-abrechnung Der Beck GmbH Filial...  1002423109   
         500  Kartenzahlung/-abrechnung MUELLER SAGT DANKE//...  1002423109   
         501  Kartenzahlung/-abrechnung DECATHLON Deuts

In [20]:
bc.update_data(df_update)

DEBUG:root:updating dataset_name: dkb_3109


ffff Index(['date', 'amount', 'desc', 'account_nr', 'target account', 'class'], dtype='object')


In [24]:
bc.save_data()

In [21]:
bc.data_unlabeled

date  amount  \
dkb_3109 477 2022-09-30 -315.64   
         478 2022-09-30   -4.00   
         491 2022-09-19  -18.59   
         495 2022-09-12  -34.00   
         496 2022-09-12  -26.00   
         498 2022-09-09   -5.69   
         499 2022-09-08   -3.95   
         501 2022-09-07  -65.95   
         504 2022-09-06  -22.59   
         505 2022-09-06  -12.60   
         506 2022-09-06   -4.90   
         508 2022-09-05  -23.00   
         514 2022-08-30  -39.00   
         515 2022-08-30   -9.30   
         518 2022-08-29   -7.00   
         522 2022-08-24  -38.00   
         523 2022-08-24   -3.80   
         528 2022-08-22  -56.58   
         531 2022-08-19  150.00   
         534 2022-08-15  -55.08   
         536 2022-08-15  -29.05   
         539 2022-08-08 -116.01   

                                                           desc    account_nr  \
dkb_3109 477  Kartenzahlung/-abrechnung VIVA RESORT PROT.PRI...  1.002423e+09   
         478  Kartenzahlung/-abrechnung H.RESORT SUPERMERCAD...  1.002423e+09   
         491  Kartenzahlung/-abrechnung ASIL GMBH//BERLIN/DE...  1.002423e+09   
         495  Kartenzahlung/-abrechnung VINOTRIA GMBH CO KG/...  1.002423e+09   
         496  Kartenzahlung/-abrechnung BEZIRKSAMT CHARLOTTE...  1.002423e+09   
         498  Kartenzahlung/-abrechnung Burger King 15800//N...  1.002423e+09   
         499  Kartenzahlung/-abrechnung Der Beck GmbH Filial...  1.002423e+09   
         501  Kartenzahlung/-abrechnung DECATHLON Deutschlan...  1.002423e+09   
         504  Kartenzahlung/-abrechnung Kiez Markt GIR 69038...  1.002423e+09   
         505  Kartenzahlung/-abrechnung DENN?S BIOMARKT BER/...  1.002423e+09   
         506  Kartenzahlung/-abrechnung DENN?S BIOMARKT BER/...  1.002423e+09   
         508  Lastschrift GIRASOLES - SONNENBLUMEN Beitrag N...  1.002423e+09   
         514  Kartenzahlung/-abrechnung MARIA BONITA//BERLIN...  1.002423e+09   
         515  Kartenzahlung/-abrechnung IZ .GIMME GELATO G//...  1.002423e+09   
         518  Kartenzahlung/-abrechnung BRLO Gastro. Holding...  1.002423e+09   
         522  Kartenzahlung/-abrechnung Photo Zentrum//Berli...  1.002423e+09   
         523  Kartenzahlung/-abrechnung SumUp .vanille maril...  1.002423e+09   
         528  Kartenzahlung/-abrechnung SB Tank 7784// Hoyer...  1.002423e+09   
         531  Gutschrift CORNELIA GIESELER UND DR. S TEFFEN ...  1.002423e+09   
         534  Lastschrift Rundfunk ARD, ZDF, DRadio Rundfunk...  1.002423e+09   
         536  Kartenzahlung/-abrechnung ROSSMANN VIELEN DANK...  1.002423e+09   
         539  Kartenzahlung/-abrechnung AMBROSINO SUPERMARKT...  1.002423e+09   

             target account class  
dkb_3109 477            NaN   NaN  
         478            NaN   NaN  
         491            NaN   NaN  
         495            NaN   NaN  
         496            NaN   NaN  
         498            NaN   NaN  
         499            NaN   NaN  
         501            NaN   NaN  
         504            NaN   NaN  
         505            NaN   NaN  
         506            NaN   NaN  
         508            NaN   NaN  
         514            NaN   NaN  
         515            NaN   NaN  
         518            NaN   NaN  
         522            NaN   NaN  
         523            NaN   NaN  
         528            NaN   NaN  
         531            NaN   NaN  
         534            NaN   NaN  
         536            NaN   NaN  
         539            NaN   NaN

### define the regex expressions and save to disk

In [22]:
import yaml

In [23]:
regex_dict = {
    r'^Lastschrift Vodafone .*': 'Bill - Communications',
    r'^Lohn, Gehalt, Rente IAV GmbH *.': 'Income - Salary & Benefits',
    r'^Lastschrift European Bank for Financial Services *.':'Investments',
    r'^Lastschrift Sauren SICAV SPARPLAN *.':'Investments',
    r'^Überweisung GISELA SILVA *.':'Education',
    r'^Dauerauftrag GIRASOLES-FOERDERVEREIN *.':'Education',
    r'^Lastschrift GIRASOLES - SONNENBLUMEN Beitrag .*':'Education',
    r'^Lastschrift VATTENFALL EUROPE *.': 'Bill - Utilities, Rent & Fees',
    r'^Lastschrift Aberdeen Standard Investments *.':'Bill - Utilities, Rent & Fees',
    r'^Kartenzahlung/-abrechnung BARES GbR *.':'Restaurants & Bars',
    r'^Kartenzahlung/-abrechnung EDEKA *.': 'Supermarket & Everyday commodities',
    r'^Kartenzahlung/-abrechnung *.ROSSMANN *.':'Supermarket & Everyday commodities',
    r'^Kartenzahlung/-abrechnung MUELLER*.': 'Supermarket & Everyday commodities',
    r'^Kartenzahlung/-abrechnung ALDI SAGT DANKE *.' : 'Supermarket & Everyday commodities',
    r'^Kartenzahlung/-abrechnung DENN?S BIOMARKT *.' : 'Supermarket & Everyday commodities',
    r'.*apotheke.*': 'Personal Care & Sport',
}


with open(Path('../data/labeling_regex.yaml'), 'w') as file:
    documents = yaml.dump(regex_dict, file)




date  amount  \
dkb_3109 492  2022-09-19  -11.94   
         493  2022-09-19   -5.00   
dkb_4217 26   2020-12-11  -66.75   
         140  2020-10-01  -27.00   
         155  2020-09-21   -7.95   
         330  2021-10-26   -5.45   
         340  2021-10-25  -32.57   
         343  2021-10-25   -6.50   
         364  2021-10-11  -13.51   
         370  2021-10-11   -2.49   
         428  2021-09-08  -17.36   
         623  2021-06-11  -10.85   
         655  2021-05-25  -27.00   
         676  2021-05-10   -6.15   
         687  2021-04-26  -46.45   
         742  2021-02-22  -32.90   
         857  2022-07-25   -8.50   
         889  2022-07-12 -147.82   
         954  2022-06-07   -5.00   
         1105 2022-03-15   -6.15   
         1154 2022-02-21  -22.90   
         1193 2022-01-31   -3.93   

                                                            desc  account_nr  \
dkb_3109 492   Kartenzahlung/-abrechnung Atlantis-Apotheke//B...  1002423109   
         493   Kartenzahlung/-abrechnung BONG APOTHEKE//BERLI...  1002423109   
dkb_4217 26    Kartenzahlung/-abrechnung Apotheke Carmer-7//B...  1071944217   
         140   Kartenzahlung/-abrechnung Apotheke Carmer-7//B...  1071944217   
         155   Kartenzahlung/-abrechnung APOTHEKE AM KLAUSENE...  1071944217   
         330   Kartenzahlung/-abrechnung MEDIOSAPOTHEKE//BERL...  1071944217   
         340   Kartenzahlung/-abrechnung APOTHEKE Q205 GIR 69...  1071944217   
         343   Kartenzahlung/-abrechnung Park-Apotheke Berit ...  1071944217   
         364   Kartenzahlung/-abrechnung GALENUS APOTHEKE . B...  1071944217   
         370   Kartenzahlung/-abrechnung GALENUS APOTHEKE . B...  1071944217   
         428   Kartenzahlung/-abrechnung GORKI APOTHEKE//BERL...  1071944217   
         623   Kartenzahlung/-abrechnung GORKI APOTHEKE//BERL...  1071944217   
         655   Kartenzahlung/-abrechnung Apotheke Carmer-7//B...  1071944217   
         676   Kartenzahlung/-abrechnung FALKEN APOTHEKE - UR...  1071944217   
         687   Kartenzahlung/-abrechnung Apotheke Berlin Bahn...  1071944217   
         742   Kartenzahlung/-abrechnung Apotheke Carmer-7//B...  1071944217   
         857   Kartenzahlung/-abrechnung GOETHE APOTHEKE//Ber...  1071944217   
         889   Kartenzahlung/-abrechnung GORKI APOTHEKE//BERL...  1071944217   
         954   Kartenzahlung/-abrechnung Apotheke an der Oper...  1071944217   
         1105  Kartenzahlung/-abrechnung APOTHEKE AM KLAUSENE...  1071944217   
         1154  Kartenzahlung/-abrechnung BONG APOTHEKE//BERLI...  1071944217   
         1193  Kartenzahlung/-abrechnung BONG APOTHEKE//BERLI...  1071944217   

              target account                  class  
dkb_3109 492             NaN                    NaN  
         493             NaN                    NaN  
dkb_4217 26              NaN  Personal Care & Sport  
         140             NaN  Personal Care & Sport  
         155             NaN  Personal Care & Sport  
         330             NaN  Personal Care & Sport  
         340             NaN  Personal Care & Sport  
         343             NaN  Personal Care & Sport  
         364             NaN  Personal Care & Sport  
         370             NaN  Personal Care & Sport  
         428             NaN  Personal Care & Sport  
         623             NaN  Personal Care & Sport  
         655             NaN  Personal Care & Sport  
         676             NaN  Personal Care & Sport  
         687             NaN  Personal Care & Sport  
         742             NaN  Personal Care & Sport  
         857             NaN  Personal Care & Sport  
         889             NaN  Personal Care & Sport  
         954             NaN  Personal Care & Sport  
         1105            NaN  Personal Care & Sport  
         1154            NaN  Personal Care & Sport  
         1193            NaN  Personal Care & Sport

In [36]:
with open(Path('../data/labeling_regex.yaml')) as file:
    documents = yaml.full_load(file)

In [37]:
documents

{'^Lastschrift Vodafone Deutschland GmbH .*': 'Bill - Communications'}

In [39]:
target_dict = {
    r'PAYPAL (EUROPE) S.A.R.L .*': 'paypal', # filter out the paypal entries
    r'^Überweisung HERR .*': 'dkb',
    r'Umbuchung KREDITKARTEN GELDANLAGE': 'dkb',
    r'^Gutschrift J.*und C.*': 'dkb_4217',
    r'^Überweisung HERR DR.*AN.*': 'dkb_3109'
}
with open(Path('../data/target_account_regex.yaml'), 'w') as file:
    documents = yaml.dump(target_dict, file)

In [40]:
with open(Path('../data/target_account_regex.yaml')) as file:
    documents = yaml.full_load(file)
documents

{'PAYPAL (EUROPE) S.A.R.L .*': 'paypal',
 'Umbuchung KREDITKARTEN GELDANLAGE': 'dkb',
 '^Gutschrift J.*und C.*': 'dkb_4217',
 '^Überweisung HERR .*': 'dkb',
 '^Überweisung HERR DR.*AN.*': 'dkb_3109'}

### manually label data with regex

In [31]:
bc.data_all[bc.data_all['desc'].str.lower().str.match(r'.*DECATHLON.*'.lower())]

date  amount  \
dkb_3109 501 2022-09-07  -65.95   
dkb_4217 145 2020-09-28   -8.76   
         421 2021-09-10   -8.99   

                                                           desc  account_nr  \
dkb_3109 501  Kartenzahlung/-abrechnung DECATHLON Deutschlan...  1002423109   
dkb_4217 145  Kartenzahlung/-abrechnung DECATHLON DEUTSCHLAN...  1071944217   
         421  Kartenzahlung/-abrechnung DECATHLON DEUTSCHLAN...  1071944217   

             target account                  class  
dkb_3109 501            NaN  Personal Care & Sport  
dkb_4217 145            NaN  Personal Care & Sport  
         421            NaN  Personal Care & Sport

In [32]:
len(bc.data_all)

1324